- А точно нужно сортировать по вероятности?
Мне кажется, вероятность необходимо учитывать, т.к. в итоге нам надо рекомендовать пользователю то, что ему вероятнее всего понравится.

- Какую метрику использовать?
moneyprecision, precision, recall

- Сколько раз в неделю отпрпавляем рассылку?
думаю 1 раз в неделю - максимум, плюс я бы дала возможность пользователям самим указывать в настройках частоту и необходимость рассылки

- В какое время отправляем рассылку?
Обеденное время (12-13), вечернее (19-21), когда люди проводят время онлайн

- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?
Можно делать тематические рассылки (в зависимости от праздников разные товары), отслеживать по каким товарам он переходит из рассылки, спрашивать обратную связь, добавить веса для понижения нерелевантных товаров, чтобы они не попадались в выборке

- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?
Да, я бы добавляла в рассылку товары из соседних групп, например, чай и конфеты, или плед и книги, какао)). 
Можно рекомендовать вообще самые популярные или то, что покупают похожие пользователи.


In [2]:
import itertools as it

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Для поиска параметров ALS
from sklearn.model_selection import GridSearchCV

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
    
from metrics import precision_at_k, recall_at_k


%matplotlib inline

In [5]:
data = pd.read_csv('retail_train.csv')

In [6]:
data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
            inplace=True)

In [7]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [9]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

In [10]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [11]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [12]:
# Заведем фиктивный item_id
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

/Users/Olga/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [13]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [14]:
# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

In [15]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

ALS

In [16]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [17]:
def score_als_precision(model, user_ids, actual_recs, N=5):
#     model.fit(csr_matrix(user_item_matrix).T,  # На вход item-user matrix
#               show_progress=True)

#     recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
#                            user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
#                            N=5, # кол-во рекомендаций 
#                            filter_already_liked_items=False, 
#                            filter_items=None, 
#                            recalculate_user=True)
    
    recs = user_ids.apply(lambda x: get_recommendations(x, model=model, N=N))
    result = pd.DataFrame(actual_recs)
    result = result.rename(columns={'user_id': 'actual'})
    result['recs'] = recs
    score = result.apply(lambda row: precision_at_k(row['recs'], row['actual']), axis=1).mean()
    
    return score

In [18]:
rec_results = data_test.groupby('user_id')['item_id'].unique().reset_index()
rec_results.columns=['user_id', 'actual']
rec_results.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [19]:
parameters = {'factors': [32, 64, 128],
              'regularization': [0.0001, 0.001, 0.01],
              'iterations': [10, 15, 20]}

In [20]:
%%time

scores = []
for factors, reg, iters in it.product(*parameters.values()):
    model = AlternatingLeastSquares(factors=factors, 
                                    regularization=reg,
                                    iterations=iters, 
                                    calculate_training_loss=True, 
                                    num_threads=6)

    model.fit(csr_matrix(user_item_matrix).T,  # На вход item-user matrix
              show_progress=True)
    
    score = {'factors': factors,
             'regularization': reg,
             'iterations': iters,
             'score': score_als_precision(model, rec_results['user_id'], rec_results['actual'], N=5)}
    scores.append(score)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

CPU times: user 50min 1s, sys: 1min 47s, total: 51min 49s
Wall time: 27min 38s


In [22]:
pd.DataFrame(scores, index=range(len(scores))).sort_values('score', ascending=False)

,factors,regularization,iterations,score
7,32,0.0100,15,0.168854
5,32,0.0010,20,0.167777
17,64,0.0100,20,0.167483
4,32,0.0010,15,0.167287
6,32,0.0100,10,0.167287
15,64,0.0100,10,0.166308
3,32,0.0010,10,0.165622
14,64,0.0010,20,0.165622
9,64,0.0001,10,0.165524
2,32,0.0001,20,0.165524
